In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense,LSTM, Input
from tensorflow.keras.layers import Concatenate, concatenate  
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
#1. 데이터

path = '../_data/juga predict/'  
SD = pd.read_csv(path +"삼성전자.csv", thousands=',')
SD = SD.drop(range(21, 1120), axis=0)                                  # 액면분할 (893, 1120)
KD = pd.read_csv(path + "키움증권.csv", thousands=',')
KD = KD.drop(range(21, 1060), axis=0)

sx = SD.drop(['일자', '고가', '저가', '전일비', 'Unnamed: 6', '등락률', '거래량', '금액(백만)', '신용비','개인', '기관', '외인(수량)','외국계','프로그램','외인비'], axis =1)
sx = np.array(sx)
kx = KD.drop(['일자', '고가', '저가', '전일비', 'Unnamed: 6', '등락률', '거래량', '금액(백만)', '신용비','개인', '기관', '외인(수량)','외국계','프로그램','외인비'], axis =1)
kx = np.array(sx)
xx1 = SD.drop(['일자', '고가', '저가', '전일비', 'Unnamed: 6', '등락률', '거래량', '금액(백만)', '신용비','개인', '기관', '외인(수량)','외국계','프로그램','외인비'], axis =1)
xx1 = np.array(xx1)
xx2 = KD.drop(['일자', '고가', '저가', '전일비', 'Unnamed: 6', '등락률', '거래량', '금액(백만)', '신용비','개인', '기관', '외인(수량)','외국계','프로그램','외인비'], axis =1)
xx2 = np.array(xx2)


In [3]:
print(sx.shape,kx.shape)

(21, 2) (21, 2)


In [55]:
def split_xy(dataset, time_steps, y_column):
    x, y =list(),list()
    for i in range(len(dataset)): 
        x_end_number = i + time_steps       
        y_end_number = x_end_number + y_column - 1                  
    
        if y_end_number > len(dataset):
            break
        tmp_x = dataset[i:x_end_number, :-1]
        tmp_y = dataset[x_end_number-1:y_end_number, -1]
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)

In [4]:
def split_xy3(dataset, time_steps, y_column):                     # size : 몇개로 나눌 것인가
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column - 1
        
        if y_end_number > len(dataset):
            break
        tmp_x = dataset[i:x_end_number, 1:]                       # [a,b]  a= 데이터의 위치, b = 칼럼의 위치
        tmp_y = dataset[x_end_number - 1:y_end_number, 0]
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)

x1, y1 = split_xy3(sx, 3, 2)
x2, y2 = split_xy3(kx, 3, 2)

print('x값 : ',x1,'\n','y값 : ',y1)

x값 :  [[[78000]
  [77800]
  [77600]]

 [[77800]
  [77600]
  [77000]]

 [[77600]
  [77000]
  [76800]]

 [[77000]
  [76800]
  [76900]]

 [[76800]
  [76900]
  [78200]]

 [[76900]
  [78200]
  [77400]]

 [[78200]
  [77400]
  [77400]]

 [[77400]
  [77400]
  [76300]]

 [[77400]
  [76300]
  [75600]]

 [[76300]
  [75600]
  [75800]]

 [[75600]
  [75800]
  [74400]]

 [[75800]
  [74400]
  [71300]]

 [[74400]
  [71300]
  [72300]]

 [[71300]
  [72300]
  [72300]]

 [[72300]
  [72300]
  [73700]]

 [[72300]
  [73700]
  [74800]]

 [[73700]
  [74800]
  [75300]]

 [[74800]
  [75300]
  [74900]]] 
 y값 :  [[76400 76500]
 [76500 77200]
 [77200 77400]
 [77400 77400]
 [77400 78300]
 [78300 76100]
 [76100 75100]
 [75100 75600]
 [75600 73900]
 [73900 72000]
 [72000 73200]
 [73200 71700]
 [71700 73500]
 [73500 75100]
 [75100 76000]
 [76000 76000]
 [76000 73300]
 [73300 70400]]


In [57]:
print(x1,y1,x2,y2)

[] [] [] []


In [39]:
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1,
        train_size =0.7, shuffle= True, random_state = 42)
x2_train, x2_test,y2_train,y2_test = train_test_split(x2,y2,
        train_size = 0.7, shuffle = True, random_state = 42)

In [40]:
#2-1 모델1
input1 = Input(shape=(4,1))   # (70,2) 
dense1 = LSTM(32, activation='relu')(input1)
dense2 = Dense(64, activation='relu')(dense1)
dense3 = Dense(32, activation='relu')(dense2)
dense4 = Dense(16, activation='relu')(dense3)
output1 = Dense(8, activation='relu')(dense4)

#2-2 모델2
input2 = Input(shape=(4,1))   # (70,3) 
dense11 = LSTM(32, activation='relu')(input2)
dense12 = Dense(64, activation='relu')(dense11)
dense13 = Dense(32, activation='relu')(dense12)
dense14 = Dense(16, activation='relu')(dense13)
output2 = Dense(8, activation='relu')(dense14)

In [41]:
merge1 = Concatenate()([output1,output2]) 

# 2-3 output모델1
output21 = Dense(16)(merge1)
output22 = Dense(8, activation='relu')(output21)
output23 = Dense(2, activation='relu')(output22)
last_output1 = Dense(1)(output23)  # y의 열의 갯수

# 2-4 output모델2
output31 = Dense(16)(merge1)
output32 = Dense(8, activation='relu')(output31)
output33 = Dense(2, activation='relu')(output32)
last_output2 = Dense(1)(output33)

model = Model(inputs = [input1,input2], outputs= ([last_output1,last_output2]))

In [42]:
#3. 컴파일, 훈련
model.compile(loss='mae', optimizer = 'adam')
 
es = EarlyStopping(monitor='val_loss', patience=10, mode='auto',
                   verbose=1, restore_best_weights=True)
mcp = ModelCheckpoint (monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only=True,
                       filepath = '../Study/_ModelCheckPoint/kovt.hdf5')
model.fit([x1_train,x2_train], [y1_train,y2_train], epochs=1000, batch_size=2,
          validation_split=0.3)

# model = load_model ('../_test/_save/kovt3.h5')

model.save('../_test/_save/kovt4.h5')


Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 184580.3125 - dense_59_loss: 76407.9062 - dense_63_loss: 108172.3984 - val_loss: 185152.5625 - val_dense_59_loss: 77032.6953 - val_dense_63_loss: 108119.8594
Epoch 2/1000
1/1 [==============================] - 0s 17ms/step - loss: 184204.7344 - dense_59_loss: 76331.4219 - dense_63_loss: 107873.3125 - val_loss: 184814.7812 - val_dense_59_loss: 76956.0781 - val_dense_63_loss: 107858.7109
Epoch 3/1000
1/1 [==============================] - 0s 12ms/step - loss: 183867.7969 - dense_59_loss: 76255.1016 - dense_63_loss: 107612.6953 - val_loss: 184544.4375 - val_dense_59_loss: 76921.8047 - val_dense_63_loss: 107622.6406
Epoch 4/1000
1/1 [==============================] - 0s 18ms/step - loss: 183595.7812 - dense_59_loss: 76219.0000 - dense_63_loss: 107376.7891 - val_loss: 184306.3438 - val_dense_59_loss: 76896.4531 - val_dense_63_loss: 107409.8984
Epoch 5/1000
1/1 [==============================] - 0s 19ms/step - loss: 18335

In [44]:
#4. 평가, 예측
loss = model.evaluate ([x1_test, x2_test], [y1_test,y2_test], batch_size=1)
print('loss :', loss) #loss :
y1_pred, y2_pred = model.predict([x1, x2])
print('삼성예측값 : ', y1_pred[-1])
print('키움예측값 : ', y2_pred[-1])


1/1 [==============================] - 0s 14ms/step - loss: 110213.4609 - dense_59_loss: 2964.4609 - dense_63_loss: 107249.0000
loss : [110213.4609375, 2964.4609375, 107249.0]
삼성예측값 :  [76202.55]
키움예측값 :  [0.99999064]
